# Define TensorFlow Session

In [ ]:
import tensorflow as tf
from keras import backend as K

sess = tf.Session()
K.set_session(sess)

# Load VGGNet

In [ ]:
from keras.applications.vgg16 import VGG16

model = VGG16()
model.summary()

# Show VGG Graph

In [ ]:
import numpy as np
from IPython.display import display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = tf.compat.as_bytes("<stripped %d bytes>"%size)
    return strip_def
  
def rename_nodes(graph_def, rename_func):
    res_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = res_def.node.add() 
        n.MergeFrom(n0)
        n.name = rename_func(n.name)
        for i, s in enumerate(n.input):
            n.input[i] = rename_func(s) if s[0]!='^' else '^'+rename_func(s[1:])
    return res_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))
  
    iframe = """
        <iframe seamless style="width:800px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

show_graph(sess.graph)

# Use TensorFlow Tensors with VGGNet

In [ ]:
from keras.objectives import categorical_crossentropy

labels = tf.placeholder(tf.float32, shape=model.output_shape)
loss = tf.reduce_mean(categorical_crossentropy(labels, model.output))

labels, loss

# Define Data

In [ ]:
nb_class = model.output_shape[1]

X, Y = np.random.randn(1, *model.input_shape[1:]), np.zeros([1, nb_class], dtype=np.float32)
Y[0, np.random.randint(nb_class)] = 1

X.shape, Y.shape

# Define Optimizer and Initialize Variables

In [ ]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

init = tf.global_variables_initializer()
sess.run(init)

# Take SGD Step

In [ ]:
import keras.backend as K

with sess.as_default():
    l, _ = sess.run(fetches=[loss, train_step], feed_dict={model.input: X, labels: Y})
    print(l)

# Evaluate

In [ ]:
from keras.metrics import categorical_accuracy as accuracy

acc_tensor = accuracy(labels, model.output)
with sess.as_default():
    print(acc_tensor.eval(feed_dict={model.input: X, labels: Y}))